In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import mediapipe as mp
import shutil
import pandas as pd

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'mediapipe'

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilitiesimport pandas as pd

file_path = 'label.xlsx'
df = pd.read_excel(file_path)
metric = df[['Label', 'Vietnam']].to_numpy() 

In [3]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [4]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# Cách Setup:
- Tạo sẵn folder chứa để tên hành động cần thu
- Đưa file note book này vào folder đấy
- Ctrl + M để chạy hết file này (nếu chưa có thư viện thì hãy dùng lệnh py -m pip install 'tên thư viện cần cài')
# Cách thu:
- Khi code chạy sẽ hiện lên 1 màn hình chờ mình chuẩn bị có hiện dòng collection frame... -> lúc này là máy chưa thu, thời gian hiện màn hình chờ mình có thể điều khiển dài ngắn được
- Khi code chạy là màn hình sẽ mất dòng chữ xanh collection frame và bắt đầu chuyển động
- Lưu ý là 1 lần thu sẽ chạy từ đầu đến cuối hết 1 video thì nó mới dừng cho lần thu tiếp theo

In [7]:
# thu keypoint truc tiep
sequence_length = 90 # Số lượng frame mỗi video
sample = 30 # Số lượng video trong 1 hành động đó
action = 'Bông hoa' # Chọn tên của hành động cần thu
background = cv2.imread('background.jpg')

for name in metric:
    if action == name[1]:
        flag = name[0]

# Xoá bộ nhớ cho máy chạy đỡ khổ
del file_path, metric, df

In [8]:
flag

'Flower'

In [9]:
# Set mediapipe model 

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 
    cv2.imshow('Ready!!!', background)
    cv2.putText(background, 'READY!!!', (120,200), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
    cv2.waitKey(3000)     
    cv2.destroyAllWindows()  
    
    for i in range(sample):
        os.mkdir('D:/Coding/CV/final_prj/keypoint/' + flag + f'/{i}')
        npy_path = 'D:/Coding/CV/final_prj/keypoint/' + flag + f'/{i}/'   
        
        cap = cv2.VideoCapture(0)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        frame_rate = 30 
        
        out = cv2.VideoWriter(f'{i}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (frame_width, frame_height))

        for frame_num in range(sequence_length):
            # Read feed
            ret, frame = cap.read()
            
            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)
            
            # NEW Apply wait logic
            if frame_num == 0: 
                cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, f'Collecting frames for {flag} Video Number {i}', (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(1000) # Điều chỉnh tgian chờ giữa các lần thu
            else: 
                cv2.putText(image, f'Collecting frames for {flag} Video Number {i}', (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', image)
            
            # NEW Export keypoints
            keypoints = extract_keypoints(results)
            out.write(frame)
            np.save((npy_path + f'{frame_num}.npy'), keypoints)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        out.release()
        cap.release()
        cv2.destroyAllWindows()